In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import re
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer, BertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm

## Processing Claims from Liar data

In [15]:
# Load train, test, validation data
train_data = pd.read_csv("train2.tsv",sep='\t', header=None)
test_data = pd.read_csv("test2.tsv",sep='\t', header=None)
val_data = pd.read_csv("val2.tsv",sep='\t', header=None)

In [16]:
# Rename all the columns
train_data.rename({1: 'id', 2: 'label', 3: 'claims', 4: 'subject', 5: 'speaker', 6: 'job-title',
           7: 'state_info', 8: 'party_affiliation', 9: 'barely_true_counts', 10: 'false_counts',
           11: 'half_true_counts', 12: 'mostly_true_counts', 13: 'pants_on_fire_counts', 14: 'context',
           15: 'evidence'
          }, axis = 1, inplace = True)
test_data.rename({1: 'id', 2: 'label', 3: 'claims', 4: 'subject', 5: 'speaker', 6: 'job-title',
           7: 'state_info', 8: 'party_affiliation', 9: 'barely_true_counts', 10: 'false_counts',
           11: 'half_true_counts', 12: 'mostly_true_counts', 13: 'pants_on_fire_counts', 14: 'context',
           15: 'evidence'
          }, axis = 1, inplace = True)
val_data.rename({1: 'id', 2: 'label', 3: 'claims', 4: 'subject', 5: 'speaker', 6: 'job-title',
           7: 'state_info', 8: 'party_affiliation', 9: 'barely_true_counts', 10: 'false_counts',
           11: 'half_true_counts', 12: 'mostly_true_counts', 13: 'pants_on_fire_counts', 14: 'context',
           15: 'evidence'
          }, axis = 1, inplace = True)

In [22]:
# Extract the 'claims' column
claims = train_data['claims']

# Save the 'claims' column to a text file
claims.to_csv('liar_claims.txt', index=False, header=False)

### POLUSA Data Process with unreliable sources filtered out


In [1]:
import csv
import json

# Replace 'your_csv_file.csv' with the path to your CSV file
csv_file_path = './polusa_balanced/2019_1.csv'

# Replace 'text' with the name of the column containing the document text
text_column_name = 'body'

# The output JSONL file
jsonl_file_path = 'polusa2019.jsonl'

# List of unreliable source outlets to exclude
unreliable_sources = ['Breitbart']

with open(csv_file_path, mode='r', encoding='utf-8') as csv_file, \
     open(jsonl_file_path, mode='w', encoding='utf-8') as jsonl_file:
    
    reader = csv.DictReader(csv_file)
    counter = 0
    for row in reader:
        if row['outlet'] in unreliable_sources:
            # Skip the row if the outlet is in the list of unreliable sources
            continue

        if counter < 20000:
            # Extract the text content from the specified column
            text = row.get(text_column_name, '')

            # Create a JSON object and write it to the JSONL file
            json_obj = {'text': text}
            jsonl_file.write(json.dumps(json_obj) + '\n')

            counter += 1
        else:
            break

print(f"Conversion complete. JSONL file saved as '{jsonl_file_path}'")